In [2]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [4]:
import PIL
from PIL import Image
def open_img(path):
    img = PIL.Image.open(path).convert("L")
    return np.array(img) 

In [5]:
def OR(edges):
    aux = np.copy(edges)
    result = edges[0]
    for edge in edges[1:]:
        result = np.array(np.logical_or(result,edge),dtype = np.int)
    return result

In [3]:
def Emax(img, l = 3):
    Emaxs = []
    imgarr = np.copy(img)
    for i in range(0,l):
        coeffs2 = pywt.dwt2(imgarr, 'haar')
        LL, (LH, HL, HH) = coeffs2
        imgarr = LL
        Emax = np.sqrt( LH**2 + HL**2 + HH**2 )
        Emaxs.append(Emax)
    return Emaxs
def edges_points(emaxs_o,thr):
    edges = []
    emaxs = np.copy(emaxs_o)
    for emax in emaxs:
        emax[emax<thr] = 0
        emax[emax>=thr] = 1
        edges.append(emax)
    
    return edges

In [54]:
def roof_gstep(emaxs, edges_o):
    emax1,emax2,emax3 = np.copy(emaxs)
    edges = np.copy(edges_o)
  
    emax1[emax1<=emax2]=1
    emax1[emax1>emax2]=0
    
    emax2[emax2<=emax3]=1
    emax2[emax2>emax3]=0
    emax1[edges!=1]=0
    emax2[edges!=1]=0
    emax3[edges!=1]=0
    
    return np.array(np.logical_and(emax1,emax2),dtype = np.int)

def roof(emaxs, edges_o):
    emax1,emax2,emax3 = np.copy(emaxs)
    edges = np.copy(edges_o)

    emax1[emax1<=emax2]=1
    emax1[emax1>emax2]=0
    
    emax2[emax2>=emax3]=1
    emax2[emax2<emax3]=0
    emax1[edges!=1]=0
    emax2[edges!=1]=0
    emax3[edges!=1]=0
    
    return np.array(np.logical_and(emax1,emax2),dtype = np.int)

def dirac_astep(emaxs, edges_o):
    emax1,emax2,emax3 = np.copy(emaxs)
    edges = np.copy(edges_o)
    
    
    emax1[emax1>=emax2]=1
    emax1[emax1<emax2]=0
    
    emax2[emax2>=emax3]=1
    emax2[emax2<emax3]=0
    
    emax1[edges!=1]=0
    emax2[edges!=1]=0
    emax3[edges!=1]=0
    
    return np.array(np.logical_and(emax1,emax2),dtype = np.int)

def blr_edges(emax1_o,rrg_o,thr):
    emax1 =  np.copy(emax1_o)
    rrg = np.copy(rrg_o)
    
    emax1[emax1<thr]=1
    emax1[emax1>=thr]=0
    emax1[rrg!=1]=0

    return emax1

In [116]:
def blur_detection(img,thr=50):

    
    
    ##etapa 1
    emax1,emax2,emax3 = Emax(img,3)
    """
    emax3r = np.copy(emax3)
    h,w = emax3r.shape
    emax1r = cv2.resize(emax1, dsize = (w,h),interpolation = cv2.INTER_NEAREST)
    emax2r = cv2.resize(emax2, dsize = (w,h),interpolation = cv2.INTER_NEAREST)
    """

    emax1r = np.copy(emax1)
    h,w = emax1r.shape
    emax2r = cv2.resize(emax2, dsize = (w,h),interpolation = cv2.INTER_NEAREST)
    emax3r = cv2.resize(emax3, dsize = (w,h),interpolation = cv2.INTER_NEAREST)
 
    ##etapa 2

    edge1,edge2,edge3 = edges_points([emax1r,emax2r,emax3r],thr)
    edges = OR([edge1,edge2,edge3])
    Nedges = np.sum(edges)

    ##etapa 3
    dirac_a = dirac_astep([emax1r,emax2r,emax3r],edges)
    Nda = np.sum(dirac_a)

    ##etapa 4

    roofe = roof([emax1r,emax2r,emax3r],edges)
    roof_g = roof_gstep([emax1r,emax2r,emax3r],edges)
    roof_roofg = OR([roofe,roof_g])
 
    Nrg = np.sum(roof_roofg)
   
    ##etapa 5
    blr = blr_edges(emax1r,roof_g,thr)
    Nbrg = np.sum(blr) 
    blrrg = roof_roofg - blr 
    Nblrg = np.sum(blrrg)
    
    
    roof_roofg = roof_roofg*255
    blr = blr*255
    blrrg = blrrg*255
  
   
    cv2.imwrite('rg.jpg',roof_roofg)
    cv2.imwrite('blr.jpg',blr)
    cv2.imwrite('blrrg.jpg',blrrg)
    #plt.imshow(blr,cmap='gray')
    
    ##etapa 6
    Per = Nda/Nedges
    
    ##etapa 7 
    BlurExtend = Nblrg/Nrg
    
    return Per,BlurExtend,Nblrg




In [140]:
import cv2
def testes(path):
    teste = open_img(path)
    testeblur = cv2.GaussianBlur(teste,(13,13),0)
    testeblur2 = cv2.GaussianBlur(testeblur,(13,13),0)

    import time

    inicial = time.time()

    per, blur_e, nfinas1 = blur_detection(teste,10)
    perb, blur_eb,nfinas2 = blur_detection(testeblur,10)
    perb3, blur_eb3,nfinas3 = blur_detection(testeblur2,5)
    final = time.time()
    tempo = final - inicial
    #print(path,nfinas2/nfinas1,nfinas3/nfinas2)
    print(path,(nfinas2/nfinas1)/(nfinas3/nfinas2))
    #print(per,blur_e,"Tempo total:",tempo)
    #print(perb,blur_eb,"Tempo total:",tempo)

In [144]:
testes('teste.jpg')
testes('teste2.jpg')
testes('teste3.jpg')
testes('teste4.jpg')
testes('teste5.jpg')
testes('teste6.jpg')
testes('teste7.jpg')
testes('teste8.jpg')
testes('teste9.jpg')

teste.jpg 0.16103799037084432
teste2.jpg 0.007908988680136854
teste3.jpg 0.26883756343608256
teste4.jpg 0.007766012804140214
teste5.jpg 0.004890310908487495
teste6.jpg 0.32247362241990285
teste7.jpg 3.0062117985039496
teste8.jpg 0.4333245991746829
teste9.jpg 1.1471121376522333


In [145]:
testes("teste10.jpg")

teste10.jpg 0.2127960316136124
